In [7]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
# For finding and loading multiple files
import os
# Use this to see how much memory the dataframes use
from sys import getsizeof
# To view keyspace.yaml
import yaml

import pseudopeople as psp

from vivarium_research_prl.utils import sizemb, MappingViaAttributes

!date
!whoami
!uname -a
!python --version
!pwd

Fri 04 Aug 2023 09:26:51 PM PDT
ndbs
Linux int-slurm-sarchive-p0002 5.4.0-135-generic #152-Ubuntu SMP Wed Nov 23 20:19:22 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux
Python 3.10.12
/mnt/share/code/ndbs/vivarium_research_prl/model_validation


# Find data

`/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop
/results/release_02_yellow/full_data/united_states_of_america/2023_07_28_08_33_09
/final_results/2023_08_02_11_38_10/pseudopeople_input_data_usa`

In [3]:
project_dir = '/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop'
model_dir = (
    f'{project_dir}/results/release_02_yellow/full_data'
    '/united_states_of_america/2023_07_28_08_33_09'
)
output_dir = f'{model_dir}/final_results/2023_08_02_11_38_10/pseudopeople_input_data_usa'

!ls -halt $output_dir

total 384K
-rw-r--r--  1 albrja IHME-Simulationscience 1.6K Aug  2 15:31 CHANGELOG.rst
drwxrwsr-x 11 albrja IHME-Simulationscience 5.0K Aug  2 15:31 .
drwxrwsr-x  2 albrja IHME-Simulationscience 167K Aug  2 15:30 taxes_dependents
drwxrwsr-x  2 albrja IHME-Simulationscience 167K Aug  2 15:28 taxes_1040
drwxrwsr-x  2 albrja IHME-Simulationscience 167K Aug  2 15:23 taxes_w2_and_1099
drwxrwsr-x  2 albrja IHME-Simulationscience 167K Aug  2 15:15 social_security
drwxrwsr-x  2 albrja IHME-Simulationscience 167K Aug  2 15:15 women_infants_and_children
drwxrwsr-x  2 albrja IHME-Simulationscience 167K Aug  2 15:15 current_population_survey
drwxrwsr-x  2 albrja IHME-Simulationscience 167K Aug  2 15:15 american_community_survey
drwxrwsr-x  2 albrja IHME-Simulationscience 167K Aug  2 15:15 decennial_census
drwxrwsr-x  2 albrja IHME-Simulationscience 334K Aug  2 14:28 logs
drwxrwsr-x  3 albrja IHME-Simulationscience  512 Aug  2 11:38 ..


In [4]:
!ls -halt $model_dir

total 1.2M
drwxrwsr-x  5 sbachmei IHME-Simulationscience 2.5K Aug  2 15:31 ..
drwxrwsr-x  5 sbachmei IHME-Simulationscience 2.5K Aug  2 15:31 final_results
drwxrwsr-x  6 sbachmei IHME-Simulationscience 5.0K Jul 31 08:59 .
-rw-rw-r--  1 sbachmei IHME-Simulationscience 1.2M Jul 29 06:01 output.hdf
drwxrwsr-x 10 sbachmei IHME-Simulationscience 4.0K Jul 28 17:09 raw_results
drwxrwsr-x  2 sbachmei IHME-Simulationscience  512 Jul 28 08:33 __pycache__
-rw-rw-r--  1 sbachmei IHME-Simulationscience  110 Jul 28 08:33 settings.py
-rw-rw-r--  1 sbachmei IHME-Simulationscience 1.9K Jul 28 08:33 model_specification.yaml
-rw-rw-r--  1 sbachmei IHME-Simulationscience   50 Jul 28 08:33 branches.yaml
-rw-rw-r--  1 sbachmei IHME-Simulationscience 2.4K Jul 28 08:33 keyspace.yaml
-rw-rw-r--  1 sbachmei IHME-Simulationscience 4.2K Jul 28 08:33 requirements.txt
drwxrwsr-x  3 sbachmei IHME-Simulationscience  512 Jul 28 08:33 logs


In [5]:
!ls -halt $model_dir/raw_results

total 268K
drwxrwsr-x  6 sbachmei IHME-Simulationscience 5.0K Jul 31 08:59 ..
drwxrwsr-x  2 sbachmei IHME-Simulationscience 167K Jul 29 06:00 taxes_dependents
drwxrwsr-x  2 sbachmei IHME-Simulationscience 167K Jul 29 05:57 taxes_1040
drwxrwsr-x  2 sbachmei IHME-Simulationscience 167K Jul 29 05:49 taxes_w2_and_1099
drwxrwsr-x  2 sbachmei IHME-Simulationscience 167K Jul 29 05:38 social_security
drwxrwsr-x  2 sbachmei IHME-Simulationscience 167K Jul 29 05:37 women_infants_and_children
drwxrwsr-x  2 sbachmei IHME-Simulationscience 167K Jul 29 05:37 current_population_survey
drwxrwsr-x  2 sbachmei IHME-Simulationscience 167K Jul 29 05:37 american_community_survey
drwxrwsr-x  2 sbachmei IHME-Simulationscience 167K Jul 29 05:37 decennial_census
drwxrwsr-x 10 sbachmei IHME-Simulationscience 4.0K Jul 28 17:09 .


# Try reading `output.hdf`

Ok, that's not helpful. I guess I need the `raw_results` directory.

In [8]:
output = pd.read_hdf(f'{model_dir}/output.hdf')
output

,run_configuration.run_key.input_draw,run_configuration.run_key.random_seed,randomness.random_seed,randomness.additional_seed,input_data.input_draw_number,placeholder_branch_name.scenario,run_configuration.run_id,run_configuration.results_directory,run_configuration.run_key.placeholder_branch_name.scenario
0,602,3595,3595,602,602,baseline,5294549d-8386-4a6d-b2c5-1e09427869b3_169055840...,/mnt/team/simulation_science/priv/engineering/...,baseline
1,602,7344,7344,602,602,baseline,0e1c1c77-e946-4e19-885d-2695ed4bb18f_169055840...,/mnt/team/simulation_science/priv/engineering/...,baseline
2,602,4123,4123,602,602,baseline,968ed4a3-17d3-41d3-8c6a-58bfcd71fb08_169055840...,/mnt/team/simulation_science/priv/engineering/...,baseline
3,602,6554,6554,602,602,baseline,5bf0bd30-5e4c-4c47-ad1d-23bf0cd08eb4_169055840...,/mnt/team/simulation_science/priv/engineering/...,baseline
4,602,4561,4561,602,602,baseline,fcbab64d-0f80-400b-a5f4-af63d680e799_169055840...,/mnt/team/simulation_science/priv/engineering/...,baseline
...,...,...,...,...,...,...,...,...,...
329,602,6118,6118,602,602,baseline,dfd98ef8-3ffa-4b11-8e54-0561f500b17d_169055840...,/mnt/team/simulation_science/priv/engineering/...,baseline
330,602,4904,4904,602,602,baseline,f61f134d-4d86-4ab0-96cb-aa8c7fa16028_169055840...,/mnt/team/simulation_science/priv/engineering/...,baseline
331,602,7164,7164,602,602,baseline,407b9654-455a-402c-9b46-64ec5ff7514c_169055840...,/mnt/team/simulation_science/priv/engineering/...,baseline
332,602,8509,8509,602,602,baseline,7b05b1a1-1c4d-4535-9ef7-8dc1ac4690e8_169055840...,/mnt/team/simulation_science/priv/engineering/...,baseline


# Try reading some raw results

In [10]:
!ls -halt $model_dir/raw_results/taxes_w2_and_1099

total 322G
-rw-rw-r--  1 sbachmei IHME-Simulationscience 988M Jul 29 05:49 taxes_w2_and_1099_2863.hdf
drwxrwsr-x  2 sbachmei IHME-Simulationscience 167K Jul 29 05:49 .
-rw-rw-r--  1 sbachmei IHME-Simulationscience 988M Jul 29 05:24 taxes_w2_and_1099_3305.hdf
-rw-rw-r--  1 sbachmei IHME-Simulationscience 987M Jul 29 04:47 taxes_w2_and_1099_5501.hdf
-rw-rw-r--  1 sbachmei IHME-Simulationscience 986M Jul 29 04:43 taxes_w2_and_1099_9066.hdf
-rw-rw-r--  1 sbachmei IHME-Simulationscience 988M Jul 29 04:26 taxes_w2_and_1099_3684.hdf
-rw-rw-r--  1 sbachmei IHME-Simulationscience 988M Jul 29 02:56 taxes_w2_and_1099_6817.hdf
-rw-rw-r--  1 sbachmei IHME-Simulationscience 987M Jul 29 02:55 taxes_w2_and_1099_2700.hdf
-rw-rw-r--  1 sbachmei IHME-Simulationscience 988M Jul 29 01:23 taxes_w2_and_1099_6776.hdf
-rw-rw-r--  1 sbachmei IHME-Simulationscience 987M Jul 29 01:06 taxes_w2_and_1099_1007.hdf
-rw-rw-r--  1 sbachmei IHME-Simulationscience 988M Jul 29 01:01 taxes_w2_and_1099_781.hdf
-rw-rw-r--  1 

## Pick 3 seeds, define directories, and read some raw results

In [35]:
seeds = [2863, 3305, 5501]
raw_w2_dir = f'{model_dir}/raw_results/taxes_w2_and_1099'
w2_dir = f'{output_dir}/taxes_w2_and_1099'

In [13]:
raw_w2 = {}
for seed in seeds:
    %time raw_w2[seed] = pd.read_hdf(f'{raw_w2_dir}/taxes_w2_and_1099_{seed}.hdf')
    print(sizemb(raw_w2[seed]), 'MB')
raw_w2.keys()

CPU times: user 1min 24s, sys: 3.67 s, total: 1min 28s
Wall time: 1min 28s
3820.817098 MB
CPU times: user 1min 24s, sys: 3.22 s, total: 1min 27s
Wall time: 1min 27s
3822.111832 MB
CPU times: user 1min 24s, sys: 3.61 s, total: 1min 28s
Wall time: 1min 28s
3816.673462 MB


dict_keys([2863, 3305, 5501])

In [14]:
raw_w2[seeds[0]]

,household_id,first_name_id,middle_name_id,last_name_id,age,copy_age,date_of_birth,copy_date_of_birth,sex,has_ssn,...,employer_id,employer_name,employer_address_id,employer_state_id,employer_puma,wages,housing_type,tax_year,race_ethnicity,tax_form
simulant_id,,,,,,,,,,,,,,,,,,,,,
0,6,0,0,0,39.743763,9.724795,1980-04-03,2010-04-10,Female,True,...,3000,not implemented,3000,42,2902,16451,Standard,2019,AIAN,W2
0,6,0,0,0,39.743763,9.724795,1980-04-03,2010-04-10,Female,True,...,1060507,not implemented,1060507,48,2800,4385,Standard,2019,AIAN,W2
1,6,1,1,0,18.998224,39.743763,2000-12-31,2010-04-10,Female,True,...,1231142,not implemented,1231142,45,1102,10396,Standard,2019,Latino,W2
5,7,5,5,4,75.963004,50.807926,1944-01-14,1969-03-10,Female,True,...,756278,not implemented,756278,34,2104,108186,Standard,2019,White,W2
6,8,6,6,6,83.873738,NaN,1936-02-15,NaT,Male,False,...,1342722,not implemented,1342722,35,200,280871,Standard,2019,Latino,W2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385703,866982,1385703,1385703,1385702,57.635764,61.587881,1983-05-15,1979-06-01,Male,True,...,1420434,not implemented,5033795,42,3202,4161,Standard,2040,White,W2
1385704,866983,1385704,1385704,1385704,44.334878,32.486167,1996-09-01,2008-07-07,Male,True,...,351338,not implemented,351338,32,401,9520,Standard,2040,Asian,W2
1385706,866983,1385706,1385706,1385706,32.486167,44.334878,2008-07-07,1996-09-01,Male,True,...,770841,not implemented,770841,48,4638,1871,Standard,2040,Asian,W2


In [15]:
raw_w2[seeds[0]].dtypes

household_id                    int64
first_name_id                   int64
middle_name_id                  int64
last_name_id                    int64
age                           float64
copy_age                      float64
date_of_birth          datetime64[ns]
copy_date_of_birth     datetime64[ns]
sex                          category
has_ssn                          bool
ssn_id                          int64
copy_ssn                      float64
address_id                      int64
state_id                        int64
puma                            int64
po_box                          int64
employer_id                     int64
employer_name                category
employer_address_id             int64
employer_state_id               int64
employer_puma                   int64
wages                           int64
housing_type                 category
tax_year                        int64
race_ethnicity               category
tax_form                     category
dtype: objec

# Concatenate 3 shards into a single dataframe

In [21]:
%time raw_w2_df = pd.concat(raw_w2, names=['seed', 'simulant_id'])
raw_w2_df

CPU times: user 7.4 s, sys: 5.69 s, total: 13.1 s
Wall time: 13.1 s


household_id  first_name_id  middle_name_id  last_name_id  \
seed simulant_id                                                              
2863 0                       6              0               0             0   
     0                       6              0               0             0   
     1                       6              1               1             0   
     5                       7              5               5             4   
     6                       8              6               6             6   
...                        ...            ...             ...           ...   
5501 1384460            868225        1384460         1384460       1384458   
     1384464            868226        1384464         1384464       1384464   
     1384466            868228        1384466         1384466       1384466   
     1384467            868228        1384467         1384467       1384466   
     1384469            868229        1384469         1384469       1384468   

                        age   copy_age date_of_birth copy_date_of_birth  \
seed simulant_id                                                          
2863 0            39.743763   9.724795    1980-04-03         2010-04-10   
     0            39.743763   9.724795    1980-04-03         2010-04-10   
     1            18.998224  39.743763    2000-12-31         2010-04-10   
     5            75.963004  50.807926    1944-01-14         1969-03-10   
     6            83.873738        NaN    1936-02-15                NaT   
...                     ...        ...           ...                ...   
5501 1384460      18.789255   7.012057    2022-03-19         2002-06-02   
     1384464      44.789357        NaN    1996-03-19                NaT   
     1384466      72.001483  75.613166    1968-12-31         1965-05-22   
     1384467      75.613166  72.001483    1965-05-22         1968-12-31   
     1384469      26.143378  25.610065    2014-11-10         2015-05-24   

                     sex  has_ssn  ...  employer_id    employer_name  \
seed simulant_id                   ...                                 
2863 0            Female     True  ...         3000  not implemented   
     0            Female     True  ...      1060507  not implemented   
     1            Female     True  ...      1231142  not implemented   
     5            Female     True  ...       756278  not implemented   
     6              Male    False  ...      1342722  not implemented   
...                  ...      ...  ...          ...              ...   
5501 1384460      Female     True  ...      1045115  not implemented   
     1384464        Male     True  ...      1650240  not implemented   
     1384466        Male    False  ...       363503  not implemented   
     1384467      Female     True  ...       400519  not implemented   
     1384469      Female     True  ...      1727679  not implemented   

                  employer_address_id  employer_state_id  employer_puma  \
seed simulant_id                                                          
2863 0                           3000                 42           2902   
     0                        1060507                 48           2800   
     1                        1231142                 45           1102   
     5                         756278                 34           2104   
     6                        1342722                 35            200   
...                               ...                ...            ...   
5501 1384460                  4777383                  6           3763   
     1384464                  4475255                 51          59307   
     1384466                  5518478                 48           5305   
     1384467                  5052543                  6           6103   
     1384469                  4902286                 47           1800   

                   wages  housing_type tax_year  race_ethnicity  tax_form  
seed simulant_id         

In [22]:
raw_w2_df.dtypes

household_id                    int64
first_name_id                   int64
middle_name_id                  int64
last_name_id                    int64
age                           float64
copy_age                      float64
date_of_birth          datetime64[ns]
copy_date_of_birth     datetime64[ns]
sex                          category
has_ssn                          bool
ssn_id                          int64
copy_ssn                      float64
address_id                      int64
state_id                        int64
puma                            int64
po_box                          int64
employer_id                     int64
employer_name                category
employer_address_id             int64
employer_state_id               int64
employer_puma                   int64
wages                           int64
housing_type                 category
tax_year                        int64
race_ethnicity               category
tax_form                     category
dtype: objec

In [23]:
%time print(sizemb(raw_w2_df), 'MB')

11305.990814 MB
CPU times: user 1.04 s, sys: 960 ms, total: 2 s
Wall time: 2 s


# Find employer columns and filter to a single employer

Hmm, employer ID 3000 also has address ID 3000. Coincidence? Nope, looks like all employer address IDs start off the same as the employer ID.

In [24]:
raw_w2_df.filter(like='employer')

employer_id    employer_name  employer_address_id  \
seed simulant_id                                                      
2863 0                   3000  not implemented                 3000   
     0                1060507  not implemented              1060507   
     1                1231142  not implemented              1231142   
     5                 756278  not implemented               756278   
     6                1342722  not implemented              1342722   
...                       ...              ...                  ...   
5501 1384460          1045115  not implemented              4777383   
     1384464          1650240  not implemented              4475255   
     1384466           363503  not implemented              5518478   
     1384467           400519  not implemented              5052543   
     1384469          1727679  not implemented              4902286   

                  employer_state_id  employer_puma  
seed simulant_id                                    
2863 0                           42           2902  
     0                           48           2800  
     1                           45           1102  
     5                           34           2104  
     6                           35            200  
...                             ...            ...  
5501 1384460                      6           3763  
     1384464                     51          59307  
     1384466                     48           5305  
     1384467                      6           6103  
     1384469                     47           1800  

[65859745 rows x 5 columns]

In [25]:
emp3000_raw = raw_w2_df.query("employer_id==3000")
emp3000_raw

household_id  first_name_id  middle_name_id  last_name_id  \
seed simulant_id                                                              
2863 0                       6              0               0             0   
     141736              56902         141736          141736        141736   
     317386             127669         317386          317386        317386   
     465919             187274         465919          465919        465919   
     810372             325612         810372          810372        810372   
...                        ...            ...             ...           ...   
5501 606917             532090         606917          606917        606915   
     606917             532090         606917          606917        606915   
     606917             532090         606917          606917        606915   
     192961                  5         192961          192961        192961   
     606917                  1         606917          606917        606915   

                        age   copy_age date_of_birth copy_date_of_birth  \
seed simulant_id                                                          
2863 0            39.743763   9.724795    1980-04-03         2010-04-10   
     141736       39.472968  34.060440    1980-07-10         1980-06-09   
     317386       28.159227  50.526978    1991-11-03         1969-06-21   
     465919       49.210425        NaN    1970-10-15                NaT   
     810372       80.997250  57.860873    1939-01-01         1962-02-19   
...                     ...        ...           ...                ...   
5501 606917       42.694196        NaN    1994-03-31                NaT   
     606917       43.690774        NaN    1994-03-31                NaT   
     606917       44.687352        NaN    1994-03-31                NaT   
     192961       77.690373        NaN    1962-03-29                NaT   
     606917       45.683929        NaN    1994-03-31                NaT   

                     sex  has_ssn  ...  employer_id    employer_name  \
seed simulant_id                   ...                                 
2863 0            Female     True  ...         3000  not implemented   
     141736         Male     True  ...         3000  not implemented   
     317386         Male     True  ...         3000  not implemented   
     465919         Male     True  ...         3000  not implemented   
     810372       Female     True  ...         3000  not implemented   
...                  ...      ...  ...          ...              ...   
5501 606917         Male     True  ...         3000  not implemented   
     606917         Male     True  ...         3000  not implemented   
     606917         Male     True  ...         3000  not implemented   
     192961         Male    False  ...         3000  not implemented   
     606917         Male     True  ...         3000  not implemented   

                  employer_address_id  employer_state_id  employer_puma  \
seed simulant_id                                                          
2863 0                           3000                 42           2902   
     141736                      3000                 42           2902   
     317386                      3000                 42           2902   
     465919                      3000                 42           2902   
     810372                      3000                 42           2902   
...                               ...                ...            ...   
5501 606917                   4489254                  4            204   
     606917                   4489254                  4            204   
     606917                   4489254                  4            204   
     192961                   4489254                  4            204   
     606917                   4489254                  4            204   

                   wages             housing_type tax_year  race_ethnicity  \
seed simulant_id       

In [26]:
emp3000_raw.filter(regex="employer|year")

employer_id    employer_name  employer_address_id  \
seed simulant_id                                                      
2863 0                   3000  not implemented                 3000   
     141736              3000  not implemented                 3000   
     317386              3000  not implemented                 3000   
     465919              3000  not implemented                 3000   
     810372              3000  not implemented                 3000   
...                       ...              ...                  ...   
5501 606917              3000  not implemented              4489254   
     606917              3000  not implemented              4489254   
     606917              3000  not implemented              4489254   
     192961              3000  not implemented              4489254   
     606917              3000  not implemented              4489254   

                  employer_state_id  employer_puma  tax_year  
seed simulant_id                                              
2863 0                           42           2902      2019  
     141736                      42           2902      2019  
     317386                      42           2902      2019  
     465919                      42           2902      2019  
     810372                      42           2902      2019  
...                             ...            ...       ...  
5501 606917                       4            204      2036  
     606917                       4            204      2037  
     606917                       4            204      2038  
     192961                       4            204      2039  
     606917                       4            204      2039  

[138 rows x 6 columns]

# Now check whether address IDs for employer 3000 are the same across shards for each year

Yep, looks like they are. That suggests that the bug must be in post-processing, assuming the bug still exists in the post-processed data.

In [28]:
emp3000_raw.query("tax_year==2019").filter(regex="employer|year")

employer_id    employer_name  employer_address_id  \
seed simulant_id                                                      
2863 0                   3000  not implemented                 3000   
     141736              3000  not implemented                 3000   
     317386              3000  not implemented                 3000   
     465919              3000  not implemented                 3000   
     810372              3000  not implemented                 3000   
3305 80812               3000  not implemented                 3000   
     867951              3000  not implemented                 3000   
5501 529760              3000  not implemented                 3000   

                  employer_state_id  employer_puma  tax_year  
seed simulant_id                                              
2863 0                           42           2902      2019  
     141736                      42           2902      2019  
     317386                      42           2902      2019  
     465919                      42           2902      2019  
     810372                      42           2902      2019  
3305 80812                       42           2902      2019  
     867951                      42           2902      2019  
5501 529760                      42           2902      2019

In [29]:
emp3000_raw.query("tax_year==2020").filter(regex="employer|year")

employer_id    employer_name  employer_address_id  \
seed simulant_id                                                      
2863 141736              3000  not implemented                 3000   
     317386              3000  not implemented                 3000   
     465919              3000  not implemented                 3000   
     615954              3000  not implemented                 3000   
     783125              3000  not implemented                 3000   
     810372              3000  not implemented                 3000   
3305 867951              3000  not implemented                 3000   
5501 411982              3000  not implemented                 3000   

                  employer_state_id  employer_puma  tax_year  
seed simulant_id                                              
2863 141736                      42           2902      2020  
     317386                      42           2902      2020  
     465919                      42           2902      2020  
     615954                      42           2902      2020  
     783125                      42           2902      2020  
     810372                      42           2902      2020  
3305 867951                      42           2902      2020  
5501 411982                      42           2902      2020

In [31]:
emp3000_raw.query("tax_year==2030").filter(regex="employer|year")

employer_id    employer_name  employer_address_id  \
seed simulant_id                                                      
2863 79469               3000  not implemented              3009883   
3305 300268              3000  not implemented              3009883   
     607021              3000  not implemented              3009883   
     617531              3000  not implemented              3009883   
     814710              3000  not implemented              3009883   
5501 28430               3000  not implemented              3009883   

                  employer_state_id  employer_puma  tax_year  
seed simulant_id                                              
2863 79469                       36           3107      2030  
3305 300268                      36           3107      2030  
     607021                      36           3107      2030  
     617531                      36           3107      2030  
     814710                      36           3107      2030  
5501 28430                       36           3107      2030

In [30]:
emp3000_raw.query("tax_year==2040").filter(regex="employer|year")

employer_id    employer_name  employer_address_id  \
seed simulant_id                                                      
2863 453606              3000  not implemented              4489254   
     1347870             3000  not implemented              4489254   
3305 128625              3000  not implemented              4489254   
     472047              3000  not implemented              4489254   
     824021              3000  not implemented              4489254   
     958853              3000  not implemented              4489254   

                  employer_state_id  employer_puma  tax_year  
seed simulant_id                                              
2863 453606                       4            204      2040  
     1347870                      4            204      2040  
3305 128625                       4            204      2040  
     472047                       4            204      2040  
     824021                       4            204      2040  
     958853                       4            204      2040

# Now load post-processed data to verify that address bug still exists

In [36]:
w2 = {}
for seed in seeds:
    %time w2[seed] = pd.read_parquet(f'{w2_dir}/taxes_w2_and_1099_{seed}.parquet')
    print(sizemb(w2[seed]), 'MB')
w2.keys()

CPU times: user 15.8 s, sys: 6.85 s, total: 22.6 s
Wall time: 14.5 s
3342.920092 MB
CPU times: user 16 s, sys: 6.87 s, total: 22.8 s
Wall time: 14.6 s
3344.251539 MB
CPU times: user 16 s, sys: 7.33 s, total: 23.3 s
Wall time: 15.6 s
3340.026684 MB


dict_keys([2863, 3305, 5501])

In [37]:
w2[seeds[0]]

,first_name,age,mailing_address_street_number,employer_street_name,ssn,tax_year,employer_zipcode,copy_date_of_birth,employer_id,tax_form,...,simulant_id,date_of_birth,household_id,mailing_address_street_name,copy_age,employer_state,middle_initial,copy_ssn,employer_city,mailing_address_city
0,Angelina,39,1210,edgecliff ct,567-07-1954,2019,18064,2010-04-10,3000,W2,...,2863_0,1980-04-03,2863_6,stanford road,9,PA,E,371-60-7764,annville,oxon hill
1,Angelina,39,1210,n 52nd st,567-07-1954,2019,76943,2010-04-10,1060507,W2,...,2863_0,1980-04-03,2863_6,stanford road,9,TX,E,371-60-7764,houston,oxon hill
2,Angel,18,1210,skyview ter,544-45-0658,2019,29577,2010-04-10,1231142,W2,...,2863_1,2000-12-31,2863_6,stanford road,39,SC,D,567-07-1954,hilton head island,oxon hill
3,Beverly,75,2459,ince dr,363-75-8209,2019,08034,1969-03-10,756278,W2,...,2863_5,1944-01-14,2863_7,village drive,50,NJ,E,213-24-6093,cherry hill,cathedral city
4,Charles,83,20,stoney crk cir,505-06-6522,2019,87413,NaT,1342722,W2,...,2863_6,1936-02-15,2863_8,fort dummer park dr,NaN,NM,F,NaN,albuquerque,lakeway
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21958701,Armando,57,8566,chaucer ave,034-74-3305,2040,19152,1979-06-01,1420434,W2,...,2863_1385703,1983-05-15,2863_866982,gables run,61,PA,S,670-02-4123,bloomfield twp,statesville
21958702,Austin,44,234,brightsand ct,166-35-0128,2040,89149,2008-07-07,351338,W2,...,2863_1385704,1996-09-01,2863_866983,nth turner avenue,32,NV,C,786-09-1363,reno,savannah
21958703,Bryan,32,234,juanita wy,786-09-1363,2040,77099,1996-09-01,770841,W2,...,2863_1385706,2008-07-07,2863_866983,nth turner avenue,44,TX,C,664-59-2258,houston,savannah
21958704,Grace,37,5923,e redfield dr,206-96-3448,2040,33161,2030-04-08,1508563,W2,...,2863_1385708,2003-10-28,2863_866984,molalla ave,10,FL,C,545-41-5272,boynton beach,atlanta


In [38]:
w2[seeds[0]].dtypes

first_name                             category
age                                       int64
mailing_address_street_number          category
employer_street_name                   category
ssn                                    category
tax_year                                  int64
employer_zipcode                       category
copy_date_of_birth               datetime64[ns]
employer_id                               int64
tax_form                               category
employer_unit_number                   category
mailing_address_po_box                  float64
employer_street_number                 category
mailing_address_state                  category
wages                                     int64
mailing_address_zipcode                category
mailing_address_unit_number            category
last_name                              category
employer_name                          category
simulant_id                            category
date_of_birth                    datetim

# Filter to employer 3000 and concatenate

Don't concatenate the 3 entire dataframes because categoricals get converted to strings, making the memory blow up.

(Ok, well I _do_ concatenate them eventually below after setting each of their categories to the union to avoid the dtype conversion.)

In [41]:
%%time
emp3000 = pd.concat(
    {seed: df.query("employer_id==3000") for seed, df in w2.items()},
    names=['seed', 'shard_index']
)
emp3000

CPU times: user 3.93 s, sys: 840 ms, total: 4.77 s
Wall time: 4.75 s


first_name  age mailing_address_street_number  \
seed shard_index                                                 
2863 0             Angelina   39                          1210   
     133181         Matthew   39                           NaN   
     299120           Tyler   28                           400   
     439018           Brian   49                           264   
     764768       Charlotte   80                           540   
...                     ...  ...                           ...   
5501 17407297       Dominic   42                           808   
     18413939       Dominic   43                           808   
     19420233       Dominic   44                           808   
     20063454     Frederick   77                           223   
     20427830       Dominic   45                          1851   

                 employer_street_name          ssn  tax_year employer_zipcode  \
seed shard_index                                                                
2863 0                   edgecliff ct  567-07-1954      2019            18064   
     133181              edgecliff ct  204-11-3457      2019            18064   
     299120              edgecliff ct  307-17-5667      2019            18064   
     439018              edgecliff ct  256-11-9605      2019            18064   
     764768              edgecliff ct  447-78-9995      2019            18064   
...                               ...          ...       ...              ...   
5501 17407297          vallejo street  565-88-4458      2036            85704   
     18413939          vallejo street  565-88-4458      2037            85704   
     19420233          vallejo street  565-88-4458      2038            85704   
     20063454          vallejo street  004-99-3656      2039            85704   
     20427830          vallejo street  565-88-4458      2039            85704   

                 copy_date_of_birth  employer_id tax_form  ...  simulant_id  \
seed shard_index                                           ...                
2863 0                   2010-04-10         3000       W2  ...       2863_0   
     133181              1980-06-09         3000       W2  ...  2863_141736   
     299120              1969-06-21         3000       W2  ...  2863_317386   
     439018                     NaT         3000       W2  ...  2863_465919   
     764768              1962-02-19         3000       W2  ...  2863_810372   
...                             ...          ...      ...  ...          ...   
5501 17407297                   NaT         3000       W2  ...  5501_606917   
     18413939                   NaT         3000       W2  ...  5501_606917   
     19420233                   NaT         3000       W2  ...  5501_606917   
     20063454                   NaT         3000       W2  ...  5501_192961   
     20427830                   NaT         3000     1099  ...  5501_606917   

                  date_of_birth household_id mailing_address_street_name  \
seed shard_index                                                           
2863 0               1980-04-03       2863_6               stanford road   
     133181          1980-07-10   2863_56902               palo alto ave   
     299120          1991-11-03  2863_127669                  w water st   
     439018          1970-10-15  2863_187274                   hubbel st   
     764768          1939-01-01  2863_325612             sw lodestone dr   
...                         ...          ...                         ...   
5501 17407297        1994-03-31  5501_532090                 s quince st   
     18413939        1994-03-31  5501_532090                 s quince st   
     19420233        1994-03-31  5501_532090                 s quince st   
     20063454        1962-03-29       5501_5         mountain quail road   
     20427830        1994-03-31       5501_1           linn station road   

                  copy_age employer_state middle_initial     copy_ssn  \
seed shard_ind

# Check same years as before

In [45]:
emp3000.query("tax_year==2019").filter(regex="simulant|employer|year").set_index('simulant_id',append=True)

employer_street_name  tax_year employer_zipcode  \
seed shard_index simulant_id                                                   
2863 0           2863_0              edgecliff ct      2019            18064   
     133181      2863_141736         edgecliff ct      2019            18064   
     299120      2863_317386         edgecliff ct      2019            18064   
     439018      2863_465919         edgecliff ct      2019            18064   
     764768      2863_810372         edgecliff ct      2019            18064   
3305 75781       3305_80812              w 1175 n      2019            18045   
     819346      3305_867951             w 1175 n      2019            18045   
5501 500787      5501_529760            e 10th st      2019            18064   

                              employer_id employer_unit_number  \
seed shard_index simulant_id                                     
2863 0           2863_0              3000                  NaN   
     133181      2863_141736         3000                  NaN   
     299120      2863_317386         3000                  NaN   
     439018      2863_465919         3000                  NaN   
     764768      2863_810372         3000                  NaN   
3305 75781       3305_80812          3000                  NaN   
     819346      3305_867951         3000                  NaN   
5501 500787      5501_529760         3000                  NaN   

                             employer_street_number          employer_name  \
seed shard_index simulant_id                                                 
2863 0           2863_0                         NaN  Ben Hicks & Save More   
     133181      2863_141736                    NaN  Ben Hicks & Save More   
     299120      2863_317386                    NaN  Ben Hicks & Save More   
     439018      2863_465919                    NaN  Ben Hicks & Save More   
     764768      2863_810372                    NaN  Ben Hicks & Save More   
3305 75781       3305_80812                    2655  Ben Hicks & Save More   
     819346      3305_867951                   2655  Ben Hicks & Save More   
5501 500787      5501_529760                    311  Ben Hicks & Save More   

                             employer_state employer_city  
seed shard_index simulant_id                               
2863 0           2863_0                  PA      annville  
     133181      2863_141736             PA      annville  
     299120      2863_317386             PA      annville  
     439018      2863_465919             PA      annville  
     764768      2863_810372             PA      annville  
3305 75781       3305_80812              PA    pittsburgh  
     819346      3305_867951             PA    pittsburgh  
5501 500787      5501_529760             PA       blawnox

In [46]:
emp3000.query("tax_year==2020").filter(regex="simulant|employer|year").set_index('simulant_id',append=True)

employer_street_name  tax_year employer_zipcode  \
seed shard_index simulant_id                                                   
2863 1085980     2863_141736         edgecliff ct      2020            18064   
     1250223     2863_317386         edgecliff ct      2020            18064   
     1389297     2863_465919         edgecliff ct      2020            18064   
     1529940     2863_615954         edgecliff ct      2020            18064   
     1686416     2863_783125         edgecliff ct      2020            18064   
     1711868     2863_810372         edgecliff ct      2020            18064   
3305 1763915     3305_867951             w 1175 n      2020            18045   
5501 1339833     5501_411982            e 10th st      2020            18064   

                              employer_id employer_unit_number  \
seed shard_index simulant_id                                     
2863 1085980     2863_141736         3000                  NaN   
     1250223     2863_317386         3000                  NaN   
     1389297     2863_465919         3000                  NaN   
     1529940     2863_615954         3000                  NaN   
     1686416     2863_783125         3000                  NaN   
     1711868     2863_810372         3000                  NaN   
3305 1763915     3305_867951         3000                  NaN   
5501 1339833     5501_411982         3000                  NaN   

                             employer_street_number          employer_name  \
seed shard_index simulant_id                                                 
2863 1085980     2863_141736                    NaN  Ben Hicks & Save More   
     1250223     2863_317386                    NaN  Ben Hicks & Save More   
     1389297     2863_465919                    NaN  Ben Hicks & Save More   
     1529940     2863_615954                    NaN  Ben Hicks & Save More   
     1686416     2863_783125                    NaN  Ben Hicks & Save More   
     1711868     2863_810372                    NaN  Ben Hicks & Save More   
3305 1763915     3305_867951                   2655  Ben Hicks & Save More   
5501 1339833     5501_411982                    311  Ben Hicks & Save More   

                             employer_state employer_city  
seed shard_index simulant_id                               
2863 1085980     2863_141736             PA      annville  
     1250223     2863_317386             PA      annville  
     1389297     2863_465919             PA      annville  
     1529940     2863_615954             PA      annville  
     1686416     2863_783125             PA      annville  
     1711868     2863_810372             PA      annville  
3305 1763915     3305_867951             PA    pittsburgh  
5501 1339833     5501_411982             PA       blawnox

In [47]:
emp3000.query("tax_year==2030").filter(regex="simulant|employer|year").set_index('simulant_id',append=True)

employer_street_name  tax_year employer_zipcode  \
seed shard_index simulant_id                                                   
2863 10870962    2863_79469           debonair ln      2030            10801   
3305 11071416    3305_300268     e madison avenue      2030            10708   
     11362177    3305_607021     e madison avenue      2030            10708   
     11372147    3305_617531     e madison avenue      2030            10708   
     11558580    3305_814710     e madison avenue      2030            10708   
5501 10805422    5501_28430        beachpoint way      2030            10805   

                              employer_id employer_unit_number  \
seed shard_index simulant_id                                     
2863 10870962    2863_79469          3000                  NaN   
3305 11071416    3305_300268         3000                  NaN   
     11362177    3305_607021         3000                  NaN   
     11372147    3305_617531         3000                  NaN   
     11558580    3305_814710         3000                  NaN   
5501 10805422    5501_28430          3000                  NaN   

                             employer_street_number          employer_name  \
seed shard_index simulant_id                                                 
2863 10870962    2863_79469                   15665  Ben Hicks & Save More   
3305 11071416    3305_300268                   1003  Ben Hicks & Save More   
     11362177    3305_607021                   1003  Ben Hicks & Save More   
     11372147    3305_617531                   1003  Ben Hicks & Save More   
     11558580    3305_814710                   1003  Ben Hicks & Save More   
5501 10805422    5501_28430                     119  Ben Hicks & Save More   

                             employer_state employer_city  
seed shard_index simulant_id                               
2863 10870962    2863_79469              NY     callicoon  
3305 11071416    3305_300268             NY     salamanca  
     11362177    3305_607021             NY     salamanca  
     11372147    3305_617531             NY     salamanca  
     11558580    3305_814710             NY     salamanca  
5501 10805422    5501_28430              NY      new york

In [48]:
emp3000.query("tax_year==2040").filter(regex="simulant|employer|year").set_index('simulant_id',append=True)

employer_street_name  tax_year employer_zipcode  \
seed shard_index simulant_id                                                    
2863 21328311    2863_453606             texas ave      2040            85749   
     21947998    2863_1347870            texas ave      2040            85749   
3305 21053837    3305_128625           zinzer raod      2040            85750   
     21349523    3305_472047           zinzer raod      2040            85750   
     21653055    3305_824021           zinzer raod      2040            85750   
     21768970    3305_958853           zinzer raod      2040            85750   

                               employer_id employer_unit_number  \
seed shard_index simulant_id                                      
2863 21328311    2863_453606          3000                  NaN   
     21947998    2863_1347870         3000                  NaN   
3305 21053837    3305_128625          3000                  NaN   
     21349523    3305_472047          3000                  NaN   
     21653055    3305_824021          3000                  NaN   
     21768970    3305_958853          3000                  NaN   

                              employer_street_number          employer_name  \
seed shard_index simulant_id                                                  
2863 21328311    2863_453606                    1332  Ben Hicks & Save More   
     21947998    2863_1347870                   1332  Ben Hicks & Save More   
3305 21053837    3305_128625                    2107  Ben Hicks & Save More   
     21349523    3305_472047                    2107  Ben Hicks & Save More   
     21653055    3305_824021                    2107  Ben Hicks & Save More   
     21768970    3305_958853                    2107  Ben Hicks & Save More   

                              employer_state employer_city  
seed shard_index simulant_id                                
2863 21328311    2863_453606              AZ       phoenix  
     21947998    2863_1347870             AZ       phoenix  
3305 21053837    3305_128625              AZ          mesa  
     21349523    3305_472047              AZ          mesa  
     21653055    3305_824021              AZ          mesa  
     21768970    3305_958853              AZ          mesa

# Check whether address IDs match across shards for _all_ employers

In [50]:
raw_w2_df.employer_id.nunique()

1699207

In [59]:
%%time
address_id_counts = raw_w2_df.groupby(['employer_id', 'tax_year'])['employer_address_id'].nunique()
address_id_counts

CPU times: user 47.5 s, sys: 3.55 s, total: 51 s
Wall time: 51 s


employer_id  tax_year
1            2019        1
             2020        1
             2021        1
             2022        1
             2023        1
                        ..
1745105      2032        1
             2037        1
             2038        1
             2039        1
             2040        1
Name: employer_address_id, Length: 23897764, dtype: int64

In [60]:
(address_id_counts == 1).all()

True

# Now try to efficiently concatenate post-processed files so I can check addresses in those

In [62]:
d = {1: 'a', 2: 'i', 7: '4'}
list(d.values())

['a', 'i', '4']

In [68]:
def conform_categories_for_iterable(shards): # Operates in place...
    """Take an iterable of dataframes with identical columns, and 
    """
    shards = list(shards)
    categorical_cols = [col for col in shards[0] if shards[0][col].dtype == 'category']
    for col in categorical_cols:
        categories = shards[0][col].cat.categories
        for shard in shards[1:]:
            categories = categories.union(shard[col].cat.categories)
        for shard in shards:
            shard[col] = shard[col].cat.set_categories(categories)
    return shards

## Copy my data to avoid reloading in case I mess something up

In [65]:
%%time
w2_copy = {seed: df.copy() for seed, df in w2.items()}
w2_copy.keys()

CPU times: user 1.96 s, sys: 3.09 s, total: 5.05 s
Wall time: 5.05 s


dict_keys([2863, 3305, 5501])

In [67]:
w2_copy[seeds[0]].first_name

0           Angelina
1           Angelina
2              Angel
3            Beverly
4            Charles
              ...   
21958701     Armando
21958702      Austin
21958703       Bryan
21958704       Grace
21958705      Tamara
Name: first_name, Length: 21958706, dtype: category
Categories (12368, object): ['Aaden', 'Aadhya', 'Aadi', 'Aaditya', ..., 'Zymir', 'Zyon', 'Zyonna', 'Zyra']

In [70]:
%%time
shards = conform_categories_for_iterable(w2_copy.values())
len(shards)

CPU times: user 22.5 s, sys: 5.72 s, total: 28.2 s
Wall time: 28.2 s


3

In [72]:
%%time
for df in shards:
    print(sizemb(df, 'MB'))

4173.538238525391
4174.382697105408
4170.835653305054
CPU times: user 3.41 s, sys: 208 ms, total: 3.61 s
Wall time: 3.61 s


In [74]:
%%time
for seed, df in w2_copy.items():
    print(seed, sizemb(df, 'MB'))

2863 4173.538238525391
3305 4174.382697105408
5501 4170.835653305054
CPU times: user 3.47 s, sys: 55.4 ms, total: 3.52 s
Wall time: 3.52 s


In [78]:
%%time
for seed, df in w2.items():
    print(seed, sizemb(df, 'MB'))

2863 3188.0570335388184
3305 3189.3268003463745
5501 3185.297664642334
CPU times: user 1.52 s, sys: 104 ms, total: 1.63 s
Wall time: 1.62 s


## Now concatenate

In [75]:
%time w2_df = pd.concat(w2_copy, names=['seed', 'shard_index'])
w2_df

CPU times: user 21.6 s, sys: 5.24 s, total: 26.8 s
Wall time: 26.8 s


first_name  age mailing_address_street_number  \
seed shard_index                                                 
2863 0             Angelina   39                          1210   
     1             Angelina   39                          1210   
     2                Angel   18                          1210   
     3              Beverly   75                          2459   
     4              Charles   83                            20   
...                     ...  ...                           ...   
5501 21934887        Gianna   18                          7612   
     21934888        Justin   44                          1110   
     21934889     Christian   72                           104   
     21934890       Frances   75                           104   
     21934891       Eleanor   26                         10368   

                 employer_street_name          ssn  tax_year employer_zipcode  \
seed shard_index                                                                
2863 0                   edgecliff ct  567-07-1954      2019            18064   
     1                      n 52nd st  567-07-1954      2019            76943   
     2                    skyview ter  544-45-0658      2019            29577   
     3                        ince dr  363-75-8209      2019            08034   
     4                 stoney crk cir  505-06-6522      2019            87413   
...                               ...          ...       ...              ...   
5501 21934887               donley rd  229-38-5753      2040            90807   
     21934888           drake acrs rd  056-36-3693      2040            20170   
     21934889                 reba dr  414-79-9273      2040            78726   
     21934890            edwardian ct  414-79-9273      2040            95650   
     21934891               encina st  809-20-6586      2040            37748   

                 copy_date_of_birth  employer_id tax_form  ...   simulant_id  \
seed shard_index                                           ...                 
2863 0                   2010-04-10         3000       W2  ...        2863_0   
     1                   2010-04-10      1060507       W2  ...        2863_0   
     2                   2010-04-10      1231142       W2  ...        2863_1   
     3                   1969-03-10       756278       W2  ...        2863_5   
     4                          NaT      1342722       W2  ...        2863_6   
...                             ...          ...      ...  ...           ...   
5501 21934887            2002-06-02      1045115       W2  ...  5501_1384460   
     21934888                   NaT      1650240       W2  ...  5501_1384464   
     21934889            1965-05-22       363503       W2  ...  5501_1384466   
     21934890            1968-12-31       400519       W2  ...  5501_1384467   
     21934891            2015-05-24      1727679       W2  ...  5501_1384469   

                  date_of_birth household_id mailing_address_street_name  \
seed shard_index                                                           
2863 0               1980-04-03       2863_6               stanford road   
     1               1980-04-03       2863_6               stanford road   
     2               2000-12-31       2863_6               stanford road   
     3               1944-01-14       2863_7               village drive   
     4               1936-02-15       2863_8         fort dummer park dr   
...                         ...          ...                         ...   
5501 21934887        2022-03-19  5501_868225                    dylan st   
     21934888        1996-03-19  5501_868226                   w 72nd pl   
     21934889        1968-12-31  5501_868228              sitio frontera   
     21934890        1965-05-22  5501_868228              sitio frontera   
     21934891        2014-11-10  5501_868229              stem creek trl   

                  copy_age employer_state middle_initial     copy_ssn  \
s

In [76]:
w2_df.dtypes

first_name                             category
age                                       int64
mailing_address_street_number          category
employer_street_name                   category
ssn                                    category
tax_year                                  int64
employer_zipcode                       category
copy_date_of_birth               datetime64[ns]
employer_id                               int64
tax_form                               category
employer_unit_number                   category
mailing_address_po_box                  float64
employer_street_number                 category
mailing_address_state                  category
wages                                     int64
mailing_address_zipcode                category
mailing_address_unit_number            category
last_name                              category
employer_name                          category
simulant_id                            category
date_of_birth                    datetim

In [77]:
print(sizemb(w2_df), 'MB') # 10.6 GB

10646.588947 MB


# Now try counting address components per employer in the post-processed data

In [56]:
w2[seeds[0]].query("employer_id==1").filter(regex='employer|year')

,employer_street_name,tax_year,employer_zipcode,employer_id,employer_unit_number,employer_street_number,employer_name,employer_state,employer_city
110,lake garden dr,2019,45409,1,NaN,7013,Military,OH,lisbon
195,lake garden dr,2019,45409,1,NaN,7013,Military,OH,lisbon
553,lake garden dr,2019,45409,1,NaN,7013,Military,OH,lisbon
590,lake garden dr,2019,45409,1,NaN,7013,Military,OH,lisbon
627,lake garden dr,2019,45409,1,NaN,7013,Military,OH,lisbon
...,...,...,...,...,...,...,...,...,...
21958484,lake garden dr,2040,45409,1,NaN,7013,Military,OH,lisbon
21958486,lake garden dr,2040,45409,1,NaN,7013,Military,OH,lisbon
21958488,lake garden dr,2040,45409,1,NaN,7013,Military,OH,lisbon
21958490,lake garden dr,2040,45409,1,NaN,7013,Military,OH,lisbon


In [81]:
w2[seeds[0]].employer_unit_number.nunique()

50607

In [90]:
w2[seeds[0]].employer_unit_number.nunique(dropna=False)

50608

In [89]:
emp3000['employer_unit_number'].unique() #nunique(dropna=False)

[NaN, 'unit 310']
Categories (50607, object): ['# -1', '# -101', '# -2', '# -20011', ..., '№ 7 fl', '№ 7 fl door 711', '№ 9 fl apt # 905', '№ 9 floor unit 936']

In [79]:
w2_df.filter(like='employer').columns

Index(['employer_street_name', 'employer_zipcode', 'employer_id',
       'employer_unit_number', 'employer_street_number', 'employer_name',
       'employer_state', 'employer_city'],
      dtype='object')

In [110]:
%%time
address_counts_per_seed = (
    w2_df
    .groupby(['seed', 'employer_id', 'tax_year'])
    [['employer_street_number', 'employer_street_name', 'employer_city', 'employer_state', 'employer_zipcode']]
    .nunique(dropna=False)
)
address_counts_per_seed

CPU times: user 3min 2s, sys: 13.6 s, total: 3min 15s
Wall time: 3min 15s


employer_street_number  employer_street_name  \
seed employer_id tax_year                                                 
2863 1           2019                           1                     1   
                 2020                           1                     1   
                 2021                           1                     1   
                 2022                           1                     1   
                 2023                           1                     1   
...                                           ...                   ...   
5501 1745105     2030                           1                     1   
                 2031                           1                     1   
                 2032                           1                     1   
                 2037                           1                     1   
                 2038                           1                     1   

                           employer_city  employer_state  employer_zipcode  
seed employer_id tax_year                                                   
2863 1           2019                  1               1                 1  
                 2020                  1               1                 1  
                 2021                  1               1                 1  
                 2022                  1               1                 1  
                 2023                  1               1                 1  
...                                  ...             ...               ...  
5501 1745105     2030                  1               1                 1  
                 2031                  1               1                 1  
                 2032                  1               1                 1  
                 2037                  1               1                 1  
                 2038                  1               1                 1  

[39771537 rows x 5 columns]

In [112]:
address_counts_per_seed.nunique()

employer_street_number    1
employer_street_name      1
employer_city             1
employer_state            1
employer_zipcode          1
dtype: int64

In [113]:
(address_counts_per_seed == 1).all(axis=None)

True

## Check whether the seed to address function is one-to-one

In [116]:
small_df = w2_df.iloc[:10_000]
small_df

first_name  age mailing_address_street_number  \
seed shard_index                                                 
2863 0             Angelina   39                          1210   
     1             Angelina   39                          1210   
     2                Angel   18                          1210   
     3              Beverly   75                          2459   
     4              Charles   83                            20   
...                     ...  ...                           ...   
     9995             Angie   51                           822   
     9996             Angie   51                           822   
     9997            Willie   69                            35   
     9998            Willie   69                            35   
     9999           Charles   69                            35   

                     employer_street_name          ssn  tax_year  \
seed shard_index                                                   
2863 0                       edgecliff ct  567-07-1954      2019   
     1                          n 52nd st  567-07-1954      2019   
     2                        skyview ter  544-45-0658      2019   
     3                            ince dr  363-75-8209      2019   
     4                     stoney crk cir  505-06-6522      2019   
...                                   ...          ...       ...   
     9995                         grv cir  560-06-4357      2019   
     9996         northwest melrose drive  560-06-4357      2019   
     9997                   washington st  197-12-6855      2019   
     9998                crystal lakes dr  197-12-6855      2019   
     9999                    bolack dr ne  033-75-3079      2019   

                 employer_zipcode copy_date_of_birth  employer_id tax_form  \
seed shard_index                                                             
2863 0                      18064         2010-04-10         3000       W2   
     1                      76943         2010-04-10      1060507       W2   
     2                      29577         2010-04-10      1231142       W2   
     3                      08034         1969-03-10       756278       W2   
     4                      87413                NaT      1342722       W2   
...                           ...                ...          ...      ...   
     9995                   28315         1941-06-16       630343       W2   
     9996                   34471         1941-06-16      1063824       W2   
     9997                   06051         1950-07-17       769584       W2   
     9998                   37128         1950-07-17      1567869       W2   
     9999                   33071         1950-07-06      1417566       W2   

                  ... simulant_id  date_of_birth household_id  \
seed shard_index  ...                                           
2863 0            ...      2863_0     1980-04-03       2863_6   
     1            ...      2863_0     1980-04-03       2863_6   
     2            ...      2863_1     2000-12-31       2863_6   
     3            ...      2863_5     1944-01-14       2863_7   
     4            ...      2863_6     1936-02-15       2863_8   
...               ...         ...            ...          ...   
     9995         ...  2863_10688     1968-12-19    2863_4320   
     9996         ...  2863_10688     1968-12-19    2863_4320   
     9997         ...  2863_10689     1950-07-06    2863_4321   
     9998         ...  2863_10689     1950-07-06    2863_4321   
     9999         ...  2863_10690     1950-07-17    2863_4321   

                 mailing_address_street_name  copy_age employer_state  \
seed shard_index                                                        
2863 0                         stanford road         9             PA   
     1                         stanford road         9             TX   
     2                         stanford road        39             SC   
     3                         village drive   

In [171]:
def yak(shards):
    shards = list(shards)
    categorical_cols = [col for col in shards[0] if shards[0][col].dtype == 'category']
    for col in categorical_cols:
        categories = shards[0][col].cat.categories
        # TODO: try this instead: https://pandas.pydata.org/docs/reference/api/pandas.api.types.union_categoricals.html
        for shard in shards[1:]:
            categories = categories.union(shard[col].cat.categories)
        for shard in shards:
            shard[col] = shard[col].cat.set_categories(categories)

def build_full_address(df, prefix=''):
    address_fields = ['street_number', 'street_name', 'unit_number', 'city', 'state', 'zipcode']
    address_colname = 'address'
    if prefix:
        address_fields = [f'{prefix}_{field}' for field in address_fields]
        address_colname = f'{prefix}_{address_colname}'

    def ensure_empty_string_in_cats(col):
        if col.dtype == 'category' and col.isna().any() and '' not in col.cat.categories:
            col = col.cat.add_categories('')
        return col

    address_cols = [ensure_empty_string_in_cats(df[field]) for field in address_fields]
    address = address_cols[0].fillna('').astype(str) # Need to ensure '' in categories to avoid an error here
    for col in address_cols[1:]:
        col = col.fillna('').astype(str)
        col.loc[col != ''] = ' ' + col # Only add a space in front of nonempty strings
        address += col
    address.rename(address_colname, inplace=True)
    return address

In [172]:
build_full_address(small_df, 'employer')

seed  shard_index
2863  0                                 edgecliff ct annville PA 18064
      1                                 309 n 52nd st houston TX 76943
      2              1960 skyview ter unit 170 hilton head island S...
      3                                 e ince dr cherry hill NJ 08034
      4                            stoney crk cir albuquerque NM 87413
                                           ...                        
      9995                                  46 grv cir weldon NC 28315
      9996                   162 northwest melrose drive lutz FL 34471
      9997                     2993 washington st glastonbury CT 06051
      9998                       125 crystal lakes dr memphis TN 37128
      9999                       3607 bolack dr ne clearwater FL 33071
Name: employer_address, Length: 10000, dtype: object

## Get full employer address

In [ ]:
%%time
employer_address = build_full_address(w2_df, 'employer') # This killed my kernel...
employer_address

In [131]:
s.astype(str).fillna('')

0    5.0
1    nan
2    8.0
dtype: object

In [135]:
s.astype('category')

/ihme/homes/ndbs/miniconda3/envs/pppl310/lib/python3.10/site-packages/IPython/lib/pretty.py:778: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  output = repr(obj)


0    5.0
1    NaN
2    8.0
dtype: category
Categories (2, float64): [5.0, 8.0]

In [136]:
t = pd.Series(['a', np.nan, 'b'])
t

0      a
1    NaN
2      b
dtype: object

In [153]:
t.astype('category')

0      a
1    NaN
2      b
dtype: category
Categories (2, object): ['a', 'b']

In [148]:
s.map(str)

0    5.0
1    nan
2    8.0
dtype: object

In [158]:
'a' in t.astype('category').cat.categories

True

In [160]:
'a' in t.array

True

In [92]:
%%time
address_piece_counts = (
    w2_df
    .groupby(['employer_id', 'tax_year'])
    [['employer_street_number', 'employer_street_name', 'employer_city', 'employer_state', 'employer_zipcode']]
    .nunique()
)
address_piece_counts

CPU times: user 3min 21s, sys: 10.2 s, total: 3min 32s
Wall time: 3min 32s


employer_street_number  employer_street_name  \
employer_id tax_year                                                 
1           2019                           3                     3   
            2020                           3                     3   
            2021                           3                     3   
            2022                           3                     3   
            2023                           3                     3   
...                                      ...                   ...   
1745105     2032                           1                     1   
            2037                           1                     1   
            2038                           1                     1   
            2039                           1                     1   
            2040                           1                     1   

                      employer_city  employer_state  employer_zipcode  
employer_id tax_year                                                   
1           2019                  3               1                 2  
            2020                  3               1                 2  
            2021                  3               1                 2  
            2022                  3               1                 2  
            2023                  3               1                 2  
...                             ...             ...               ...  
1745105     2032                  1               1                 1  
            2037                  1               1                 1  
            2038                  1               1                 1  
            2039                  1               1                 1  
            2040                  1               1                 1  

[23897764 rows x 5 columns]

In [93]:
%%time
seed_counts = (
    w2_df
    .reset_index('seed')
    .groupby(['employer_id', 'tax_year'])
    [['seed']]
    .nunique()
)
seed_counts

CPU times: user 34 s, sys: 7.42 s, total: 41.4 s
Wall time: 41.4 s


seed
employer_id tax_year      
1           2019         3
            2020         3
            2021         3
            2022         3
            2023         3
...                    ...
1745105     2032         1
            2037         1
            2038         1
            2039         1
            2040         1

[23897764 rows x 1 columns]

In [94]:
all_counts = address_piece_counts.join(seed_counts)
all_counts

employer_street_number  employer_street_name  \
employer_id tax_year                                                 
1           2019                           3                     3   
            2020                           3                     3   
            2021                           3                     3   
            2022                           3                     3   
            2023                           3                     3   
...                                      ...                   ...   
1745105     2032                           1                     1   
            2037                           1                     1   
            2038                           1                     1   
            2039                           1                     1   
            2040                           1                     1   

                      employer_city  employer_state  employer_zipcode  seed  
employer_id tax_year                                                         
1           2019                  3               1                 2     3  
            2020                  3               1                 2     3  
            2021                  3               1                 2     3  
            2022                  3               1                 2     3  
            2023                  3               1                 2     3  
...                             ...             ...               ...   ...  
1745105     2032                  1               1                 1     1  
            2037                  1               1                 1     1  
            2038                  1               1                 1     1  
            2039                  1               1                 1     1  
            2040                  1               1                 1     1  

[23897764 rows x 6 columns]

In [97]:
all_counts.drop(columns=['employer_state', 'employer_zipcode'])

employer_street_number  employer_street_name  \
employer_id tax_year                                                 
1           2019                           3                     3   
            2020                           3                     3   
            2021                           3                     3   
            2022                           3                     3   
            2023                           3                     3   
...                                      ...                   ...   
1745105     2032                           1                     1   
            2037                           1                     1   
            2038                           1                     1   
            2039                           1                     1   
            2040                           1                     1   

                      employer_city  seed  
employer_id tax_year                       
1           2019                  3     3  
            2020                  3     3  
            2021                  3     3  
            2022                  3     3  
            2023                  3     3  
...                             ...   ...  
1745105     2032                  1     1  
            2037                  1     1  
            2038                  1     1  
            2039                  1     1  
            2040                  1     1  

[23897764 rows x 4 columns]

In [99]:
emp3000.nunique(axis=1)

seed  shard_index
2863  0              25
      133181         25
      299120         25
      439018         22
      764768         25
                     ..
5501  17407297       23
      18413939       23
      19420233       23
      20063454       23
      20427830       23
Length: 138, dtype: int64

In [100]:
%%time
nunique_counts = all_counts.drop(columns=['employer_state', 'employer_zipcode']).nunique(axis=1)
nunique_counts

CPU times: user 10min 13s, sys: 18.2 s, total: 10min 31s
Wall time: 10min 19s


employer_id  tax_year
1            2019        1
             2020        1
             2021        1
             2022        1
             2023        1
                        ..
1745105      2032        1
             2037        1
             2038        1
             2039        1
             2040        1
Length: 23897764, dtype: int64

In [104]:
nunique_counts.value_counts()

1    21165183
2     2717705
3       14876
dtype: int64

In [102]:
nunique_counts.loc[nunique_counts != 1]

employer_id  tax_year
2            2035        2
             2036        2
3            2025        2
             2026        2
             2027        2
                        ..
1745097      2035        2
1745101      2038        2
             2039        2
             2040        2
1745103      2037        2
Length: 2732581, dtype: int64

In [103]:
all_counts.loc[nunique_counts != 1]

employer_street_number  employer_street_name  \
employer_id tax_year                                                 
2           2035                           0                     1   
            2036                           0                     1   
3           2025                           3                     3   
            2026                           3                     3   
            2027                           3                     3   
...                                      ...                   ...   
1745097     2035                           1                     2   
1745101     2038                           1                     2   
            2039                           1                     2   
            2040                           2                     3   
1745103     2037                           0                     1   

                      employer_city  employer_state  employer_zipcode  seed  
employer_id tax_year                                                         
2           2035                  1               1                 1     1  
            2036                  1               1                 1     1  
3           2025                  2               1                 2     3  
            2026                  2               1                 2     3  
            2027                  2               1                 2     3  
...                             ...             ...               ...   ...  
1745097     2035                  2               1                 2     2  
1745101     2038                  2               1                 2     2  
            2039                  2               1                 2     2  
            2040                  3               1                 3     3  
1745103     2037                  1               1                 1     1  

[2732581 rows x 6 columns]

In [106]:
emp3 = w2_df.query("employer_id==3")
emp3

first_name  age mailing_address_street_number  \
seed shard_index                                                 
2863 163088         Marilyn   48                         300 a   
     310621         Bethany   19                          4102   
     571700       Alexander   31                           132   
     583595        Virginia   36                         20601   
     896146            Evan   18                          7015   
...                     ...  ...                           ...   
5501 18124846       Michael   44                          3841   
     18635411        Robert   80                          6730   
     19137030       Michael   45                          3841   
     19637230        Robert   81                          6730   
     20640889        Robert   81                          6730   

                 employer_street_name          ssn  tax_year employer_zipcode  \
seed shard_index                                                                
2863 163088                 donna cir  227-33-7554      2019            97115   
     310621                 donna cir  543-29-5062      2019            97115   
     571700                 donna cir  128-78-7362      2019            97115   
     583595                 donna cir  831-90-6693      2019            97115   
     896146                 donna cir  327-66-2045      2019            97115   
...                               ...          ...       ...              ...   
5501 18124846              raven lane  753-21-5416      2037            07071   
     18635411              raven lane  211-05-2771      2037            07071   
     19137030          mt sterling rd  607-96-9914      2038            33612   
     19637230          mt sterling rd  211-05-2771      2038            33612   
     20640889          mt sterling rd  211-05-2771      2039            33612   

                 copy_date_of_birth  employer_id tax_form  ...  simulant_id  \
seed shard_index                                           ...                
2863 163088              1997-03-19            3       W2  ...  2863_173241   
     310621              1977-02-16            3       W2  ...  2863_329453   
     571700              1951-12-20            3       W2  ...  2863_606420   
     583595              1958-03-20            3       W2  ...  2863_618889   
     896146              2003-03-03            3       W2  ...  2863_950553   
...                             ...          ...      ...  ...          ...   
5501 18124846                   NaT            3       W2  ...  5501_291562   
     18635411            1958-11-11            3       W2  ...  5501_849268   
     19137030                   NaT            3       W2  ...  5501_291562   
     19637230            1958-11-11            3       W2  ...  5501_849268   
     20640889            1958-11-11            3       W2  ...  5501_849268   

                  date_of_birth household_id mailing_address_street_name  \
seed shard_index                                                           
2863 163088          1971-11-06   2863_69677                  mayfair ct   
     310621          2000-06-11  2863_132509      white heron bay circle   
     571700          1988-12-18  2863_243702         merlin falcon trail   
     583595          1983-05-30  2863_192794                  cleaves st   
     896146          2001-06-23  2863_381874               terrace vw ln   
...                         ...          ...                         ...   
5501 18124846        1993-03-27  5501_423647                  sheldon rd   
     18635411        1957-04-17  5501_342123            charlesgate west   
     19137030        1993-03-27  5501_423647                  sheldon rd   
     19637230        1957-04-17  5501_342123            charlesgate west   
     20640889        1957-04-17  5501_342123            charlesgate west   

                  copy_age employer_state middle_initial     copy_ssn  \
seed shard_ind

In [107]:
emp3.query("tax_year==2025").filter(regex="employer")

employer_street_name employer_zipcode  employer_id  \
seed shard_index                                                      
2863 6191155              e delta cir            19709            3   
     6688935              e delta cir            19709            3   
3305 5827686             soundview dr            19709            3   
     6012426             soundview dr            19709            3   
     6363614             soundview dr            19709            3   
     6686032             soundview dr            19709            3   
5501 6437835          montreal road w            19720            3   

                 employer_unit_number employer_street_number  \
seed shard_index                                               
2863 6191155                      NaN                  16323   
     6688935                      NaN                  16323   
3305 5827686                      NaN                     13   
     6012426                      NaN                     13   
     6363614                      NaN                     13   
     6686032                      NaN                     13   
5501 6437835                      NaN                   1431   

                      employer_name employer_state employer_city  
seed shard_index                                                  
2863 6191155      Doug Johnson Park             DE    wilmington  
     6688935      Doug Johnson Park             DE    wilmington  
3305 5827686      Doug Johnson Park             DE        newark  
     6012426      Doug Johnson Park             DE        newark  
     6363614      Doug Johnson Park             DE        newark  
     6686032      Doug Johnson Park             DE        newark  
5501 6437835      Doug Johnson Park             DE        newark

In [108]:
emp2 = w2_df.query("employer_id==2")
emp2

first_name  age mailing_address_street_number  \
seed shard_index                                                 
2863 1758325          Grant   22                          3000   
     2716621          Grant   23                         54932   
     6727438          Jaime   29                           122   
     7718376          Jaime   30                           122   
     8711384          Jaime   31                           122   
     10728867       Jameson   26                          4155   
     11732402       Jameson   27                          4155   
     12737332       Jameson   28                          4155   
3305 12634391       Vanessa   65                           318   
     13643119       Vanessa   66                           318   
5501 16346959         Jorge   34                          2850   
     17358734         Jorge   35                          8837   

                 employer_street_name          ssn  tax_year employer_zipcode  \
seed shard_index                                                                
2863 1758325                sylvan dr  594-89-6606      2020            07675   
     2716621                jaguar dr  594-89-6606      2021            89506   
     6727438          saint james ter  877-09-5536      2025            92804   
     7718376          saint james ter  877-09-5536      2026            92804   
     8711384          saint james ter  877-09-5536      2027            92804   
     10728867         saint james ter  276-15-5112      2029            92804   
     11732402         saint james ter  276-15-5112      2030            92804   
     12737332         saint james ter  276-15-5112      2031            92804   
3305 12634391                nahum dr  291-54-2478      2031            92804   
     13643119                nahum dr  291-54-2478      2032            92804   
5501 16346959             lancer rd s  646-26-1749      2035            92804   
     17358734             lancer rd s  646-26-1749      2036            92804   

                 copy_date_of_birth  employer_id tax_form  ...  simulant_id  \
seed shard_index                                           ...                
2863 1758325             1999-09-01            2       W2  ...  2863_859652   
     2716621             1999-09-01            2       W2  ...  2863_859652   
     6727438             1990-12-19            2       W2  ...  2863_970514   
     7718376             1990-12-19            2       W2  ...  2863_970514   
     8711384             1990-12-19            2       W2  ...  2863_970514   
     10728867            1983-01-20            2       W2  ...  2863_987955   
     11732402            1983-01-20            2       W2  ...  2863_987955   
     12737332            2015-09-13            2       W2  ...  2863_987955   
3305 12634391            1964-11-09            2       W2  ...  3305_889171   
     13643119            1964-11-09            2       W2  ...  3305_889171   
5501 16346959            1977-02-09            2       W2  ...  5501_554566   
     17358734            1977-02-09            2       W2  ...  5501_554566   

                  date_of_birth household_id mailing_address_street_name  \
seed shard_index                                                           
2863 1758325         1998-05-21  2863_345522                holworthy st   
     2716621         1998-05-21  2863_345522            van ostrand road   
     6727438         1996-11-30  2863_288563                  trails end   
     7718376         1996-11-30  2863_288563                  trails end   
     8711384         1996-11-30  2863_288563                  trails end   
     10728867        2003-02-27  2863_194744            west easy street   
     11732402        2003-02-27  2863_194744            west easy street   
     12737332        2003-02-27  2863_194744            west easy street   
3305 12634391        1966-09-19  3305_357718               lake side ave   
     13643119 

In [109]:
emp2.filter(regex="employer")

employer_street_name employer_zipcode  employer_id  \
seed shard_index                                                      
2863 1758325                sylvan dr            07675            2   
     2716621                jaguar dr            89506            2   
     6727438          saint james ter            92804            2   
     7718376          saint james ter            92804            2   
     8711384          saint james ter            92804            2   
     10728867         saint james ter            92804            2   
     11732402         saint james ter            92804            2   
     12737332         saint james ter            92804            2   
3305 12634391                nahum dr            92804            2   
     13643119                nahum dr            92804            2   
5501 16346959             lancer rd s            92804            2   
     17358734             lancer rd s            92804            2   

                 employer_unit_number employer_street_number  \
seed shard_index                                               
2863 1758325                      NaN                    312   
     2716621                      NaN                   3910   
     6727438                      NaN                    231   
     7718376                      NaN                    231   
     8711384                      NaN                    231   
     10728867                     NaN                    231   
     11732402                     NaN                    231   
     12737332                     NaN                    231   
3305 12634391                     NaN                    107   
     13643119                     NaN                    107   
5501 16346959                     NaN                    NaN   
     17358734                     NaN                    NaN   

                                 employer_name employer_state employer_city  
seed shard_index                                                             
2863 1758325      Exton Family Medicine Clinic             NJ      metuchen  
     2716621      Exton Family Medicine Clinic             NV          reno  
     6727438      Exton Family Medicine Clinic             CA       hayward  
     7718376      Exton Family Medicine Clinic             CA       hayward  
     8711384      Exton Family Medicine Clinic             CA       hayward  
     10728867     Exton Family Medicine Clinic             CA       hayward  
     11732402     Exton Family Medicine Clinic             CA       hayward  
     12737332     Exton Family Medicine Clinic             CA       hayward  
3305 12634391     Exton Family Medicine Clinic             CA   porterville  
     13643119     Exton Family Medicine Clinic             CA   porterville  
5501 16346959     Exton Family Medicine Clinic             CA     san diego  
     17358734     Exton Family Medicine Clinic             CA     san diego